1 Remove small rects

In [ ]:
import pdfplumber
import pandas as pd

selected_rects = []

def is_nested(rect, other_rect):
    #is_equal = other_rect['x0'] == rect['x0'] and other_rect['x1'] == rect['x1'] and other_rect['y0'] == rect['y0'] and other_rect['y1'] == rect['y1']
    contains = other_rect['x0'] <= rect['x0'] and other_rect['x1'] >= rect['x1'] and other_rect['y0'] <= rect['y0'] and other_rect['y1'] >= rect['y1']
    
    # print(f"Value of contains: {contains}")
    # print(f"other_rect['x0']: {other_rect['x0']}")
    # print(f"rect['x0']: {rect['x0']}")
    # print(f"other_rect['x1']: {other_rect['x1']}")
    # print(f"rect['x1']: {rect['x1']}")
    # print(f"other_rect['y0']: {other_rect['y0']}")
    # print(f"rect['y0']: {rect['y0']}")
    # print(f"other_rect['y1']: {other_rect['y1']}")
    # print(f"rect['y1']: {rect['y1']}")
    #return (not is_equal) and contains 
    return contains 
        # break #If other_rect contains rect, then rect is a nested rect and should be ignored

with pdfplumber.open("mvfodevarer_side12.pdf") as pdf:
    print(pdf.pages)
    page = pdf.pages[0]
    #TODO: 
    # 1. Ignore rects with height = 0? (or under certain threshold)
    # 2. Ignore nested rects
    im = page.to_image(resolution = 400)

    initial_j = 0

    for i in range(0,len(page.rects)):
    # for i in range(0,11):
    #Selected rects
        rect = page.rects[i]
        if rect['height'] < 1 or rect['width'] < 1: continue #1 Ignore small rectangles

        #Look for rectangles that might encapsulate current rectangle on page
        for j in range(initial_j,len(page.rects)):
            print(f"(i,j): ({i},{j})")
        # for j in range(0,10):
            other_rect = page.rects[j]
            if j == i: #Assumption: rectangles with index > i will never encapsulate rectangle i.
                print("added to selected rects")
                selected_rects.append(rect)
            elif is_nested(rect, other_rect):
                initial_j = j #Displace the starting point to search for rectangles that nest rectangle i.
                break
        
        print(f"Added rect with index {i}")
        # print(f"Rect index: {i}")
        # print(f"Rect height: {rect['height']}")
        # print(f"Rect width: {rect['width']}")
    im = im.draw_rects(selected_rects)
    im.show()

[<Page:1>]
(i,j): (0,0)
added to selected rects
(i,j): (0,1)
(i,j): (0,2)
(i,j): (0,3)
(i,j): (0,4)
(i,j): (0,5)
(i,j): (0,6)
(i,j): (0,7)
(i,j): (0,8)
(i,j): (0,9)
(i,j): (0,10)
(i,j): (0,11)
(i,j): (0,12)
(i,j): (0,13)
(i,j): (0,14)
(i,j): (0,15)
(i,j): (0,16)
(i,j): (0,17)
(i,j): (0,18)
(i,j): (0,19)
(i,j): (0,20)
(i,j): (0,21)
(i,j): (0,22)
(i,j): (0,23)
(i,j): (0,24)
(i,j): (0,25)
(i,j): (0,26)
(i,j): (0,27)
(i,j): (0,28)
(i,j): (0,29)
(i,j): (0,30)
(i,j): (0,31)
(i,j): (0,32)
(i,j): (0,33)
(i,j): (0,34)
(i,j): (0,35)
(i,j): (0,36)
(i,j): (0,37)
(i,j): (0,38)
(i,j): (0,39)
(i,j): (0,40)
(i,j): (0,41)
(i,j): (0,42)
(i,j): (0,43)
(i,j): (0,44)
(i,j): (0,45)
(i,j): (0,46)
(i,j): (0,47)
(i,j): (0,48)
(i,j): (0,49)
(i,j): (0,50)
(i,j): (0,51)
(i,j): (0,52)
(i,j): (0,53)
(i,j): (0,54)
(i,j): (0,55)
(i,j): (0,56)
(i,j): (0,57)
(i,j): (0,58)
(i,j): (0,59)
(i,j): (0,60)
(i,j): (0,61)
(i,j): (0,62)
(i,j): (0,63)
(i,j): (0,64)
(i,j): (0,65)
(i,j): (0,66)
(i,j): (0,67)
(i,j): (0,68)
(i,j): (0

In [62]:
#Drawing rects
selected_rects = []

with pdfplumber.open("../mvfodevarer_side12.pdf") as pdf:
    print(pdf.pages)
    page = pdf.pages[0]
    #TODO: 
    # 1. Ignore rects with height = 0? (or under certain threshold)
    # 2. Ignore nested rects
    im = page.to_image(resolution = 400)

    # for i in range(0,len(page.rects)):
    for i in range(0,20):
    #Selected rects
        rect = page.rects[i]
        # if rect['height'] < 1 or rect['width'] < 1: continue #1 Ignore small rectangles
        selected_rects.append(rect)
        # print(f"Rect index: {i}")
        # print(f"Rect height: {rect['height']}")
        # print(f"Rect width: {rect['width']}")
    im = im.draw_rects(selected_rects)
    im.show()

[<Page:1>]


2 Remove nested rects